# Importing data
Images data is imported in a csv form, however it can have several other formats like: 

 - Normal format .jpg / .png
 - Excel / Csv files
 - hdf5

In [ ]:
import pandas as pd
mnist_test = pd.read_csv("../input/mnist-in-csv/mnist_test.csv")
mnist_train = pd.read_csv("../input/mnist-in-csv/mnist_train.csv")

# Training data

 - Here we drop the label column from the X_train as we set it as training data; as we all know that the train data can't contain the trageted class on it, the y_train only contains the label column as it is the label to be predicted by the trained model.
 - The axis = 1 is a sign that we drop a column, if axis is set into 0 then we are dropping a row.

In [ ]:
X_train = mnist_train.drop(labels = ["label"],axis = 1) 
y_train = mnist_train["label"]

# Testing data
Testing data is completely seperated from trainig data as they can't be preproccessed together even if the preproccessing steps are the same, test data is considered as a data the model only sees in the final chapter of the modeling story not before that.

In [ ]:
X_test = mnist_test.drop(labels = ["label"],axis = 1) 

In [ ]:
sample_submission = pd.read_csv("../input/digit-recognizer/sample_submission.csv")

# Normalization
We do normalization step to make it easy for the model to learn and converge without heavy computations needed for big numbers, this step sets a pixel range of values to be between float 0 and 1, instead of integer 0 to 255

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

## Reshape

As we are working with a csv data, we can't forget reshaping it to the images form (-1,28,28) in our case here, (28,28 ) is the dimension of the image and it can be anything depending on your dataset, an image can be RGB colored image which contains 3 dimensions and can be grayscale, that's why we put -1 in the begginning of the reshaping so we don't have to worry about either if it's 2d or 3d

In [ ]:
X_test = X_test.values.reshape(-1,28,28,1)

In [ ]:
X_train = X_train.values.reshape(-1,28,28,1)

## One hot encoder

One hot encoding is simply turning this form (7) into this form (0,0,0,0,0,0,0,1,0,0) 
and this (2) into (0,0,1,0,0,0,0,0,0,0) .. etc

In [ ]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

y_train = to_categorical(y_train, num_classes = 10)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state=2)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
batch_size = 32

In [ ]:
history = model.fit(X_train,y_train, batch_size = batch_size, epochs = 20, validation_data = (X_val,y_val),verbose = 2)

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
Y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10)) 

In [ ]:
labels = model.predict(X_test)

# Sample Submission
Now let's create the csv submission file

In [ ]:
sample_submission

In [ ]:
sample_submission.to_csv('submission.csv', index=False)